# Hyperparameter tuning

In de reeds besproken machine learning technieken hebben we reeds een aantal keer vermeld dat er hyperparameters (denk aan regularisatieparameters, manieren van regularisatie, kernel type, ...).

In het geval van lineaire regressie gaat het dan over:
* L1 of L2 norm
* Regularisatieparameter $\lambda$
* learning rate

In het geval van SVM over:
* Type kernel
* Regularisatieparameter C
* Regularisatieparameter $\gamma$

Tot nu bestond de zoektocht naar de optimale combinatie van deze parameters door het manueel uitproberen en evalueren van een reeks combinaties van parameters.
Deze methode is echter niet schaalbaar en kan geautomatiseerd worden.
Dit gebeurd door middel van een gridsearch.

## Gridsearch

Het gridsearch algoritme bestaat eruit om een lijst op te stellen voor elke parameter welke waarden moeten getest worden.
Voor elke mogelijke combinatie van parameters gaat er dan een model getrained en geevalueerd worden.
Een voorbeeld van hoe dit kan geautomatiseerd worden binnen sklearn kan [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) gevonden worden.

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import numpy as np

iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':np.arange(1, 10, 1), "gamma": np.arange(0.1, 10, 0.1)}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
%time clf.fit(iris.data, iris.target)
clf.get_params()

De standaard methode van hierboven gaat alle combinaties afgaan.
Andere methoden die sneller maar niet alle combinaties aftoetsen zijn
* [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)
*[HalvingGridSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html#sklearn.model_selection.HalvingGridSearchCV)
*[HalvingRandomizedSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html#sklearn.model_selection.HalvingRandomSearchCV)

Belangrijk om hierbij op te merken is dat het GridSearch algoritme enkel verschillende parameters van het model trained en dat er geen eigenschappen van de data kan veranderd worden.
Indien je ook een exhausieve search wilt doen van het aantal hogere orde features of de vorm van scaling die gebruikt wordt op input parameters. Moet je een eigen wrapper schrijven die nog deze zaken uittest en de performantie van de uiteindelijke modellen vergelijkt.

## Validatieset

Welke data kunnen we nu gebruiken om deze gridsearch te evalueren.
Zowel de testdata als de trainingsdata kan niet gebruikt worden omdat we niet kunnen evalueren op de data waarmee het model getrained is.
Om deze reden wordt de dataset typisch in drie opgedeeld, namelijk een training-, test- en validatieset.
De validatieset is de data die dan gebruikt kan worden voor hyperparameter tuning.
Typisch wordt de dataset dan in de volgende groottes opgedeeld:
* Testset: 15%
* Validatieset: 15% 
* Trainingsdata: 70%

Dit zijn echter geen vaste waarden en kunnen wat verschillen in de praktijk.
Hoe meer data je beschibaar is hoe groter het percentage trainingsdata kan zijn. 
In het geval van big-data applicaties kan dit oplopen tot 98%.

## K-fold cross validation

Bij het steeds gebruiken van dezelfde validatieset is het mogelijk dat er een unieke split is die leidt tot een onverwacht goed of slecht resultaat.
Om dit tegen te gaan kan er gebruik gemaakt worden van K-fold cross validation.
Daarbij berekenen we de verwachte error K keer, elke keer met een andere train en validatie set om zo de kans te verhogen dat het uiteindelijke model ook goed werkt op de testset met ongeziene data.
Standaard wordt er bij het gebruik van het gridsearch algoritme gebruik gemaakt van 5 folds voor het zoeken naar de beste hyperparameters.
Indien de standaard manier niet voldoet voor de gewenste toepassing kan je ook de split rechtstreeks uitvoeren.
Meer informatie over deze methode vind je [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

## Oefening

In de diabetes dataset van sklearn is een hele reeks data beschikbaar over een aantal medische eigenschappen van personen en of deze personen diabetes hebben of niet.
Ga nu op zoek naar het beste model om te voorspellen of een persoon diabetes gaat hebben of niet.
Test hierbij zowel de logistische regressie en svm methoden en maak gebruik van gridsearch met 10-fold cross validation om de verschillende hyperparameters te testen. 

Wat is de hoogst behaalde accuraatheid en de benodigde hyperparameters?

Indien dit gelukt is, zoek ook het model dat de hoogste weighted f1-score behaald. 
Welke techniek gebruikte dit model en welke hyperparameters zijn er hiervoor gekozen?
Vergelijk beide modellen. Is er een significant verschil in de resulterende hyperparameters?
Is de behaalde accuraatheid sterk afwijkend?

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_diabetes

In [ ]:
df = load_diabetes(as_frame=True).frame
display(df)
y = df.target
X = df.drop("target", axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
# waarom doen we hier geen fit? Omdat we het op dezelfde manier willen scalen als de train-set
X_test = scaler.transform(X_test)

#model = LogisticRegression()
model = SVC()
# ga naar de api en kijk wat we kunnen varieren
#parameters = {'penalty':['l1', 'l2'], 'C':np.arange(0.1, 10, 0.1), 'solver':['liblinear']}
#parameters = {'C':np.arange(0.1, 10, 0.1), 'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'penalty':['l2']}
# list of dictionaries om combinaties van parameters te vermijden die niet toegelaten/nuttig zijn
parameters =  [{'C':np.arange(0.1, 10, 0.4), 'kernel':['linear']},
              {'C':np.arange(0.1, 10, 0.4), 'kernel':['rbf'], "gamma":np.arange(0.1, 1, 0.1)},
              {'C':np.arange(0.1, 10, 0.4), 'kernel':['linear'],"gamma":np.arange(0.1, 1, 0.1), "degree":np.arange(2, 4)}]

gridSearch = GridSearchCV(model, parameters, cv=5, verbose=2) # , scoring="f1_weighted"
gridSearch.fit(X_train, y_train)
gridSearch.get_params()

In [ ]:
gridSearch.best_params_
gridSearch.best_score_